In [1]:
import json
from lime import lime_tabular #pip install lime
import numpy as np
import os
import sys
import tensorflow as tf

sys.path.insert(0, os.path.join(os.getcwd(), "../Code/"))
from Data import DataManager
from ExplanationMetrics import Wrapper, generate_neighbor
from Models import MLP

# The networks are small enough that training is faster on CPU
os.environ["CUDA_VISIBLE_DEVICES"]="-1"

def unpack_coefs(explainer, x, predict_fn, num_features, x_train, num_samples = 1000):
    d = x_train.shape[1]
    coefs = np.zeros((d))

    u = np.mean(x_train, axis = 0)
    sd = np.sqrt(np.var(x_train, axis = 0))

    exp = explainer.explain_instance(x, predict_fn, num_features = num_features, num_samples = num_samples)

    coef_pairs = exp.local_exp[1]
    for pair in coef_pairs:
        coefs[pair[0]] = pair[1]

    coefs = coefs / sd

    intercept = exp.intercept[1] - np.sum(coefs * u)

    return np.insert(coefs, 0, intercept)



In [2]:
# Choose the point that we are going to explain

data = DataManager("../Datasets/housing.csv")

X_train = data.X_train

x = data.X_test[0]
n_input = x.shape[0]

print(x)

[ 0.55578371  2.45552053  1.0290794   0.92879049  1.39480671  1.03968118
 -1.15996953  1.62497316  1.5049378   0.79688198  0.4389692   0.11085877]


In [3]:
# Load the network shape

with open("../UCI-None/config.json", "r") as f:
    config_list = json.load(f)
    
config = config_list["housing"]

shape = [n_input]
for i in range(config[0]):
    shape.append(config[1])
shape.append(1)

print(shape)

[12, 100, 100, 100, 100, 100, 1]


In [4]:
# Create the model

tf.reset_default_graph()

X = tf.placeholder("float", [None, n_input], name = "X_in")

network = MLP(shape)
with tf.variable_scope("model", reuse = tf.AUTO_REUSE):
    pred = network.model(X)
    
saver = tf.train.Saver(max_to_keep = 1)

In [5]:
# Evaluate LIME on the unregularized model (copy of UCI-None/NoL2Reg/TF/housing/..../trial0)

with tf.Session() as sess:
    # Restore model
    saver.restore(sess, "./None/model.cpkt")
    
    # Wrap it for LIME
    wrapper = Wrapper(sess, pred, X)
    wrapper.set_index(0)
    
    # Configure LIME
    exp = lime_tabular.LimeTabularExplainer(X_train, discretize_continuous = False, mode = "regression")
    
    # Get the explanation for the chosen point
    coefs = unpack_coefs(exp, x, wrapper.predict_index, n_input, X_train)
    
    print("Local Explanation: ", np.round(coefs, 2))
        
    # Evaluate the fidelity metric
    fidelity = 0.0
    for i in range(10):
        x_pert = generate_neighbor(x)

        model_pred = wrapper.predict_index(x_pert.reshape(1, n_input))
        lime_pred = np.dot(np.insert(x_pert, 0, 1), coefs)
        fidelity += (lime_pred - model_pred)**2
    fidelity /= 10
                                           
    print("Fidelity Metric: ", fidelity)


INFO:tensorflow:Restoring parameters from ./None/model.cpkt
Local Explanation:  [ 0.11  0.01 -0.05  0.1  -0.23  0.22 -0.08 -0.38  0.24 -0.27 -0.11  0.12
 -0.34]
Fidelity Metric:  1.1464940983520875


In [6]:
# Load the ExpO regularized model (copy of UCI-LF1D/TF/housing/..../trial0)

with tf.Session() as sess:
    saver.restore(sess, "./ExpO/model.cpkt")
    
    # Wrap it for LIME
    wrapper = Wrapper(sess, pred, X)
    wrapper.set_index(0)
    
    # Configure LIME
    exp = lime_tabular.LimeTabularExplainer(X_train, discretize_continuous = False, mode = "regression")
    
    # Get the explanation for the chosen point
    coefs = unpack_coefs(exp, x, wrapper.predict_index, n_input, X_train)
    
    print("Local Explanation: ", np.round(coefs, 2))
    
    # Evaluate the fidelity metric
    fidelity = 0.0
    for i in range(10):
        x_pert = generate_neighbor(x)

        model_pred = wrapper.predict_index(x_pert.reshape(1, n_input))
        lime_pred = np.dot(np.insert(x_pert, 0, 1), coefs)
        fidelity += (lime_pred - model_pred)**2
    fidelity /= 10
                                           
    print("Fidelity Metric: ", fidelity)


INFO:tensorflow:Restoring parameters from ./ExpO/model.cpkt
Local Explanation:  [-0.04  0.04 -0.03 -0.01 -0.18  0.2   0.02 -0.15  0.17 -0.11 -0.14 -0.01
 -0.53]
Fidelity Metric:  0.017843739472360298
